In [8]:
import os
from pathlib import Path
import glob
import sys
from OSmOSE import Job_builder
from OSmOSE.utils import *

path_osmose_dataset = "/home/datawork-osmose/dataset/"

display_folder_storage_infos(path_osmose_dataset)

Total storage space (TB): 40.0
Used storage space (TB): 34.6
-----------------------
Available storage space (TB): 5.4


In [2]:
list_dataset(path_osmose_dataset, '')

Built datasets:
  - campaign: SPM_AuralA -- dataset: 2010
  - campaign: SPM_AuralA -- dataset: 201112
  - campaign: CETIROISE -- dataset: A2
  - campaign: CETIROISE -- dataset: A2_reshaped_60min
  - campaign: APOCADO -- dataset: APOCADO_C1D1_ST336363566
  - campaign: / -- dataset: APOCADO_C2D1_IROISE_07072022
  - campaign: / -- dataset: APOCADO_C5D9_ST7181
  - campaign: / -- dataset: APOCADO_C5D9_ST7194
  - campaign: / -- dataset: APOCADO_C6D3_ST7178
  - campaign: / -- dataset: APOCADO_C6D3_ST7180
  - campaign: / -- dataset: APOCADO_C7D6_ST7189
  - campaign: / -- dataset: APOCADO_training
  - campaign: CETIROISE -- dataset: B1
  - campaign: CETIROISE -- dataset: B1_reshaped_decimated
  - campaign: / -- dataset: BlueFinLibrary_BallenyIslands2015
  - campaign: / -- dataset: BlueFinLibrary_RossSea2014
  - campaign: / -- dataset: BlueFinLibrary_casey2014
  - campaign: / -- dataset: BlueFinLibrary_casey2017
  - campaign: / -- dataset: BlueFinLibrary_kerguelen2014
  - campaign: DELGOST -- da

In [3]:
list_dataset_name = ["APOCADO_C1D1_ST336363566"]

list_campaign_name = ["APOCADO"] * len(list_dataset_name)

In [4]:
for campaign_ID, dataset_ID in zip(list_campaign_name, list_dataset_name):
    check_available_file_resolution(path_osmose_dataset, campaign_ID, dataset_ID)


Dataset : APOCADO/APOCADO_C1D1_ST336363566
Available Resolution (LengthFile_samplerate) :
 7199_144000
 10_144000


In [5]:
dataset_resolution = '10_144000'

batch_size = 500

param_detect = {
          "model": 3,
          "threshold": 85,
          "ICI_max": 0.25,
          "sum_peaks": 8,
          "h_morph": 115,
          "l_morph": 1,
          "width_min": 6,
          "f_min": 20000,
          "f_max": 288000,
          "output_image": True,
          }

In [6]:
for campaign_ID, dataset_ID in zip(list_campaign_name, list_dataset_name):
    
    print(f'\n### {dataset_ID} ###', end='\n')
    
    dataset_path = os.path.join(path_osmose_dataset, os.path.join(campaign_ID,dataset_ID))
    data_path = os.path.join(dataset_path, 'data', 'audio', dataset_resolution)
    wav_files = sorted(glob.glob(os.path.join(data_path, '**/*.wav'), recursive=True))
    
    batch_num = (len(wav_files) + batch_size - 1) // batch_size

    print(f' -number of wav files : {len(wav_files)}')
    print(f' -batch size : {batch_size}')
    print(f' -number of jobs : {batch_num}')


### APOCADO_C1D1_ST336363566 ###
 -number of wav files : 18000
 -batch size : 500
 -number of jobs : 36


In [7]:
for campaign_ID, dataset_ID in zip(list_campaign_name, list_dataset_name):
    
    print(f'\n### {dataset_ID} ###', end='\n\n')
    
    dataset_path = os.path.join(path_osmose_dataset, os.path.join(campaign_ID,dataset_ID))
    data_path = os.path.join(dataset_path, 'data', 'audio', dataset_resolution)
    wav_files = sorted(glob.glob(os.path.join(data_path, '**/*.wav'), recursive=True))
    
    metadata = os.path.join(data_path, 'metadata.csv')
    with open(metadata, "r") as file:
        lines = file.readlines()
        if len(lines) == 2:
            UTC_offset = lines[1].split(',')[4][-5:-2]
            print(f'\tUTC offset: {UTC_offset}')
            
    batch_num = (len(wav_files) + batch_size - 1) // batch_size

    print(f'\tnumber of wav files : {len(wav_files)}')
    print(f'\tbatch size : {batch_size}')
    print(f'\tnumber of jobs : {batch_num}')
    
    campaign_ID = str(False) if campaign_ID == '' else campaign_ID
    param_all = [path_osmose_dataset,
         campaign_ID,    
         dataset_ID, 
         dataset_resolution, 
         UTC_offset,
         str(len(wav_files)),
         str(batch_size),
         str(param_detect['model']),
         str(param_detect['threshold']),
         str(param_detect['ICI_max']),
         str(param_detect['sum_peaks']),
         str(param_detect['h_morph']),
         str(param_detect['l_morph']),
         str(param_detect['width_min']),
         str(param_detect['f_min']),
         str(param_detect['f_max']),
         str(param_detect['output_image'])
        ]
    
    # run thalassa script

    ## log folder
    logdir = os.path.join(dataset_path, 'processed', 'thalassa', 'log')
    ## spectro folder
    spectro_path = os.path.join(dataset_path, 'processed', 'thalassa', 'spectro')

    if not os.path.exists(os.path.join(dataset_path, 'processed')):
        os.mkdir(os.path.join(dataset_path, 'processed'))
    if not os.path.exists(os.path.join(dataset_path, 'processed', 'thalassa')):
        os.mkdir(os.path.join(dataset_path, 'processed', 'thalassa'))
    if not os.path.exists(logdir):
        os.mkdir(logdir)
    if param_detect['output_image'] is True:
        if not os.path.exists(spectro_path):
            os.mkdir(spectro_path)

    env_script = ". /appli/anaconda/latest/etc/profile.d/conda.sh; conda activate ${env_name}/"
    env_name = sys.executable.replace('bin/python', '')
    script_name = os.path.abspath('./src/thalassa_main.py')
    reshape_job_id_list = []

    for batch_id in range(batch_num):

        idx_begin = batch_id * batch_size
        batch_wav = wav_files[idx_begin : idx_begin + batch_size]
        script_args = f"""{batch_id} {' '.join(param_all)} {' '.join(batch_wav)}"""

        jb = Job_builder()
        jobfile = jb.build_job_file(
                script_path=Path(script_name), \
                script_args=script_args,
                preset="low",
                jobname="batch_"+str(batch_id), 
                nodes=1,
                ncpus=16,
                queue='mpi_1',
                env_script = env_script,
                env_name=env_name,
                mem="1gb",
                walltime="04:00:00",
                logdir=Path(logdir))

        reshape_job_id_list += jb.submit_job(jobfile)

    ## merge detection files aftewards
    script_name = os.path.abspath('./src/merge_csv.py')
    path_csv = os.path.join(dataset_path, 'processed', 'thalassa')
    script_args = f'-i {path_csv} -n {dataset_ID}'

    jb = Job_builder()
    jb.build_job_file(
        script_path=Path(script_name),
        script_args=script_args,
        jobname="merge_csv",
        preset="low",
        mem="1gb",
        walltime="00:30:00",
        logdir=Path(logdir),
        env_script=env_script,
        env_name=env_name
    )            
    job_id = jb.submit_job(dependency=reshape_job_id_list)

    pending_jobs = job_id



### APOCADO_C1D1_ST336363566 ###

	UTC offset: +02
	number of wav files : 18000
	batch size : 500
	number of jobs : 36
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST336363566/processed/thalassa/log/batch_0_13-59-16_Torque_0.pbs
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST336363566/processed/thalassa/log/batch_1_13-59-16_Torque_1.pbs
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST336363566/processed/thalassa/log/batch_2_13-59-16_Torque_2.pbs
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST336363566/processed/thalassa/log/batch_3_13-59-16_Torque_3.pbs
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST336363566/processed/thalassa/log/batch_4_13-59-16_Torque_4.pbs
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST336363566/processed/thalassa/log/batch_5_13-59-16_Torque_5.pbs
Sent command qsub /home/datawork-osmose/dataset/APOCADO/APOCADO_C1D1_ST33636